In [1]:
import sys
import os
import pandas as pd

path = os.path.abspath('../..')
if path not in sys.path:
    sys.path.insert(0, path)
    
sys.path

from primer3plus.utils import reverse_complement as rc
import primer3
from aqbt.contrib.uwbf import primer_utils
from aqbt import AquariumBuildTools
aqtools = AquariumBuildTools.from_toml('creds.secret.toml')
aqtools.sessions

aq = aqtools.sessions['production']['aquarium']

In [11]:
samples = aq.Sample.where("id > 36000 AND user_id = 66 AND sample_type_id = {}".format(aq.SampleType.find_by_name("Fragment").id))

In [23]:
p3 = "<pGRR-W(\d+)W(\d+)-"

parts = []
for s in samples:
    m = re.search(p3, s.name)
    if m:
        parts.append(m.groups())
        
parts = list(set(parts))

In [34]:
import re
from tqdm.auto import tqdm

p1 = '(SD2) pp2-pGRR-W{i}'

p2 = '(SD2) W{i}W{j}-pGRR-ps'


name_ = '(SD2) pp2-pGRR-W{i}W{j}-ps'
PROJECT = "SD2 2021 Redesigns"

new_frags = []
for i, j in tqdm(parts):
    f1 = p1.format(i=i)
    f2 = p2.format(i=i, j=j)
    name = name_.format(i=i, j=j)
    
    frag1 = aq.Sample.find_by_name(f1)
    frag2 = aq.Sample.find_by_name(f2)
    assert frag1
    assert frag2
    
    length = frag1.properties['Length'] + frag2.properties['Length'] - 20
    
    props = {
        'id': None,
        'sample_type_id': aq.SampleType.find_by_name('Fragment').id,
        'name': name,
        'Length': int(length),
        'Template': name,
        'project': PROJECT,
        'Yeast Marker': None,
        'Forward Primer': frag1.properties['Forward Primer'].name,
        'Reverse Primer': frag2.properties['Reverse Primer'].name,
        'Fragment Mix Array': [frag1, frag2],
    }
    
    new_frag = aq.Sample.new(
        name=name,
        project=PROJECT,
        description='',
        sample_type_id=aq.SampleType.find_by_name('Fragment').id,
        properties={
            'Length': int(length),
            'Yeast Marker': None,
            'Forward Primer': frag1.properties['Forward Primer'],
            'Reverse Primer': frag2.properties['Reverse Primer'],
            'Fragment Mix Array': [frag1, frag2],
            'Sequence': ''
        }
    )
    
    new_frags.append(new_frag)
    
    

  0%|          | 0/12 [00:00<?, ?it/s]

In [35]:
for f in tqdm(new_frags):
    f.save()
    
    f_ = aq.Sample.find_by_name(f.name)
    f_.update_properties({'Template': f_})
    f_.save()

  0%|          | 0/12 [00:00<?, ?it/s]

TridentRequestError: Error response:
Name The sample name '(SD2) pp2-pGRR-W10W8-ps' is the name of an existing sample

In [ ]:
frags = aq